# Preprocessing Niederschlagsdaten ERA 5

Wegen der Datengröße mussten alle Jahre einzeln gedownloaded werden. <br>
Mit diesem Code werden sie zu einem Dataframe zusammengefügt. <br>
Die Rohdaten liegen in NetCDF-Format vor. Diese können nicht direkt aus dem GitHub-Repository geladen werden, <br>
sondern müssen erst lokal gespeichert werden und vom lokalen Laufwerk in das Projekt geladen werden. <br>
Dementsprechend müssen beim Ausführen des Codes die Dateipfade jeweils angepasst werden.

In [3]:
# benötigte Pakete laden
import pandas as pd
import xarray as xr
import glob
import os

# Daten manuell von GitHub herunterladen und
# lokale working directory setzen
work_dir_prec = r"C:/Users/henry/Documents/A git/projektstudie-murgtal/1_daten/ERA5_precipitation/raw"
os.chdir(work_dir_prec)

# alle Dateien aufrufen, die mit ".nc" enden, also alle in diesem Ordner
nc_files_prec = sorted(glob.glob(os.path.join(work_dir_prec, "*.nc")))
# Anzahl der aufgerufenen Dateien
print(f"Found {len(nc_files_prec)} NetCDF files.")


Found 43 NetCDF files.


In [4]:
# Dateien aller Jahre zu einer großen Datei zusammenfügen

# alle dataframes listen
all_data_prec = []

# Schleife über alle Dateien/Jahre
for i, years in enumerate(nc_files_prec):
    
    # print(f"Reading year {i+1}/{len(nc_files_prec)}: {os.path.basename(years)}")
    
    # dataset öffnen
    ds_prec = xr.open_dataset(years)
    
    # zu dataframe konvertieren
    df_prec = ds_prec.to_dataframe().reset_index()
    
    # dataframe listen
    all_data_prec.append(df_prec)
    
    # dataframe schließen
    ds_prec.close()

# alle dataframes aneinanderhängen
combined_df_prec = pd.concat(all_data_prec, ignore_index=True)

# nach Zeit sortieren
combined_df_prec = combined_df_prec.sort_values('valid_time').reset_index(drop=True)

# lokal als csv speichern
output_csv_prec = "C:/Users/henry/Documents/A git/projektstudie-murgtal/1_daten/ERA5_precipitation/era5_prec_daily_1981_2023.csv"
combined_df_prec.to_csv(output_csv_prec, index=False)

print(f"Saved combined data to: {output_csv_prec}")
print(f"Total rows: {len(combined_df_prec)}")
print(f"Date range: {combined_df_prec['valid_time'].min()} to {combined_df_prec['valid_time'].max()}")

Saved combined data to: C:/Users/henry/Documents/A git/projektstudie-murgtal/1_daten/ERA5_precipitation/era5_prec_daily_1981_2023.csv
Total rows: 62820
Date range: 1981-01-01 00:00:00 to 2023-12-31 00:00:00


4 Gitterzellen des ERA-5-Netzes überdecken das Murgtal. <br> Für die folgende Analyse verwenden wir nur einen Wert, den durchschnitt der vier Gitterzellen

In [ ]:
# Mittel der 4 Gitterzellen berrechnen

# Liste mit allen Niederschlagsdaten von lokalem Laufwerk einladen
input_csv_prec_concat = "C:/Users/henry/Documents/A git/projektstudie-murgtal/1_daten/ERA5_precipitation/era5_prec_daily_1981_2023.csv"
# als pandas dataframe bearbeiten
df_prec_concat = pd.read_csv(input_csv_prec_concat)

# Datemsatz nach Datum (Tag) gruppieren, Mittelwert der 4 Zeilen berrechnen
df_prec_concat_mean = df_prec_concat.groupby('valid_time')['tp'].mean().reset_index()

# Spalten umbennenen 
df_prec_concat_mean.columns = ['date', 'tp_sum']

# lokal als .csv speichern
output_csv_prec_concat = "C:/Users/henry/Documents/A git/projektstudie-murgtal/1_daten/ERA5_precipitation/era5_prec_daily_spatial_mean_1981_2023.csv"
df_prec_concat_mean.to_csv(output_csv_prec_concat, index=False)

print(f"Spatial mean data: {len(df_prec_concat_mean)} rows")
print(f"Saved to: {output_csv_prec_concat}")


Spatial mean data: 15705 rows
Saved to: C:/Users/henry/Documents/A git/projektstudie-murgtal/1_daten/ERA5_precipitation/era5_prec_daily_spatial_mean_1981_2023.csv


Die Daten liegen noch in m vor. <br>
Im letzten Bearbeitungsschritt werden sie in mm umgerechnet.

In [ ]:
# gemittelte Niederschlagsdaten von lokalem Laufwerk laden
csv_file_units = "C:/Users/henry/Documents/A git/projektstudie-murgtal/1_daten/ERA5_precipitation/era5_prec_daily_spatial_mean_1981_2023.csv"
# als pandas dataframe bearbeiten
df_units = pd.read_csv(csv_file_units)

# umrechnen
df_units['tp_sum'] = df_units['tp_sum'] * 1000

# alte Datei mit neuen Werten überschreiben
df_units.to_csv(csv_file_units, index=False)

print(f"\nUpdated {csv_file_units} with precipitation in mm")


Updated C:/Users/henry/Documents/A git/projektstudie-murgtal/1_daten/ERA5_precipitation/era5_prec_daily_spatial_mean_1981_2023.csv with precipitation in mm
